In [2]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
import sys

import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.impute import SimpleImputer

In [3]:
count = 0
first = 0
prev_date = '2009-12'
month = ['01','02','03','04','05','06','07','08','09','10','11','12']
year = list(range(2010, 2023))
year = [str(x) for x in year]

forecast_final = []

In [4]:
def get_data(date, prev_date, count, first):
    monthly_data = (pd.read_csv('Monthly/current.csv').dropna(how='all'))
    quarterly_data = (pd.read_csv('Quarterly/current.csv').dropna(how='all'))

    # Remove transform row from dataset
    transform_m = monthly_data.iloc[0, 1:]
    dataset_m = monthly_data.iloc[1:]

    transform_q = quarterly_data.iloc[1, 1:]
    factors_q = quarterly_data.iloc[0, 1:]
    dataset_q = quarterly_data.iloc[2:]

    # Extract the date as an index
    dataset_m.index = pd.PeriodIndex(dataset_m.sasdate.tolist(), freq='M')
    dataset_m.drop('sasdate', axis=1, inplace=True)

    dataset_q.index = pd.PeriodIndex(dataset_q.sasdate.tolist(), freq='Q')
    dataset_q.drop('sasdate', axis=1, inplace=True)

    dataset_m = dataset_m.loc[:date]
    dataset_q = dataset_q.loc[:prev_date]
    
    if count == 3:
        count = 0
    else:
        pass

    return transform_m, dataset_m, transform_q, factors_q, dataset_q, count, first

In [5]:
def transform(indicator, transforms):
    transformation = transforms[indicator.name]

    mult = 4 if indicator.index.freqstr[0] == 'Q' else 1
    
    # 1 -> No Transformation
    if transformation == 1:
        pass
    # 2 -> Difference
    elif transformation == 2:
        indicator = indicator.diff()
    # 3 -> Difference Squared
    elif transformation == 3:
        indicator = indicator.diff().diff()
    # 4 -> Log
    elif transformation == 4:
        indicator = np.log(indicator)
    # 5 -> Log Difference
    elif transformation == 5:
        indicator = np.log(indicator).diff() * 100 * mult
    # 6 -> Log Difference Squared
    elif transformation == 6:
        indicator = np.log(indicator).diff().diff() * 100 * mult
    # 7 -> Exact Difference
    elif transformation == 7:
        indicator = ((indicator / indicator.shift(1))**mult - 1) * 100
    
    return indicator

In [6]:
def trans_def(dataset_m, dataset_q, transform_m, transform_q):
# Run Transformation on Dataset

    transform_dataset_m = dataset_m.apply(transform, axis = 0, transforms = transform_m)
    transform_dataset_q = dataset_q.apply(transform, axis = 0, transforms = transform_q)

    return transform_dataset_m, transform_dataset_q

In [7]:
def rename_columns(transform_dataset_m, transform_dataset_q):
    appendix_md = (pd.read_csv('FRED-MD_updated_appendix.csv', encoding= 'unicode_escape').dropna(how='all'))
    appendix_qd = (pd.read_csv('FRED-QD_updated_appendix.csv', encoding= 'unicode_escape').dropna(how='all'))

    appendix_md.sort_values('group', inplace=True)

    definitions_md = {}
    definitions_qd = {}
    for i in range(0, len(appendix_md)):
        definitions_md[appendix_md['fred'][i]] = appendix_md['description'][i]
        
    for i in range(0, len(appendix_qd)):
        definitions_qd[appendix_qd['FRED'][i]] = appendix_qd['DESCRIPTION'][i]
        
    transform_dataset_m.rename(columns=definitions_md, inplace=True)
    transform_dataset_q.rename(columns=definitions_qd, inplace=True)

    transform_dataset_m = transform_dataset_m.reindex(list(appendix_md['description']), axis=1)
    transform_dataset_q = transform_dataset_q.reindex(list(appendix_qd['DESCRIPTION']), axis=1)

    return transform_dataset_m, transform_dataset_q, appendix_md, appendix_qd

In [8]:
def sort_groups(appendix_md):
    groups_md = {}
    for i in range(0, len(appendix_md)):
        groups_md[appendix_md['description'][i]] = appendix_md['group'][i]

    for i in groups_md.keys():
        if groups_md.get(i) == 1:
            groups_md[i] = 'Output and Income'
        if groups_md.get(i) == 2:
            groups_md[i] = 'Labor Market'
        if groups_md.get(i) == 3:
            groups_md[i] = 'Housing'
        if groups_md.get(i) == 4:
            groups_md[i] = 'Consumption, Orders, and Inventories'
        if groups_md.get(i) == 5:
            groups_md[i] = 'Money and Credit'
        if groups_md.get(i) == 6:
            groups_md[i] = 'Interest and Exchange Rates'
        if groups_md.get(i) == 7:
            groups_md[i] = 'Prices'
        if groups_md.get(i) == 8:
            groups_md[i] = 'Stock Market'

    groups = pd.DataFrame()
    groups['description'] = groups_md.keys()
    groups['group'] = groups_md.values()

    gdp = pd.DataFrame()
    gdp['description'] = ['Real Gross Domestic Product, 3 Decimal (Billions of Chained 2012 Dollars)']
    gdp['group'] = ['Output and Income']
    groups = groups.append({'description': 'Real Gross Domestic Product, 3 Decimal (Billions of Chained 2012 Dollars)', 'group': 'Output and Income'}, ignore_index = True)

    return groups

In [9]:
def load_factors(groups):
    factors = {}

    for i in range(0,len(groups)):
        factors[groups['description'][i]] = ['Global', groups['group'][i]]

    factor_orders = {'Global': 4}
    factor_multiplicities = {'Global': 2}

    return factors, factor_orders, factor_multiplicities

In [9]:
def run_model(transform_dataset_m, transform_dataset_q, factors, factor_orders, factor_multiplicities):
    endog_m = transform_dataset_m.loc['1984':, :]
    endog_q = transform_dataset_q.loc['1984':, 'Real Gross Domestic Product, 3 Decimal (Billions of Chained 2012 Dollars)']

    # Construct the dynamic factor model
    model = sm.tsa.DynamicFactorMQ(
        endog_m,
        endog_quarterly=endog_q,
        factors=factors,
        factor_orders=factor_orders,
        factor_multiplicities=factor_multiplicities
    )

    results = model.fit()
    return results

In [10]:
def hope(results, transform_dataset_q, date, dataset_q):

    gdp_description = 'Real Gross Domestic Product, 3 Decimal (Billions of Chained 2012 Dollars)'
    forecast = results.forecast(steps=1)[gdp_description].resample('Q').last()

    plot = pd.concat([transform_dataset_q.loc['1984': date , gdp_description], forecast])

    plot_orig = (plot / 100 + 1)**0.25
    plot_orig.loc['1984Q1'] = dataset_q.loc['1984Q1', 'GDPC1']
    plot_orig = plot_orig.cumprod()

    forecast_gdp = plot_orig[len(plot_orig)-1]

    return forecast_gdp

In [11]:
for y in year:
    for m in month:
        date = y + '-' + m
        if date == '2022-06':
            sys.exit()
        else:
            
            print(date, prev_date)
            [transform_m, dataset_m, transform_q, factors_q, dataset_q, count, first] = get_data(date, prev_date, count, first)
            [transform_dataset_m, transform_dataset_q] = trans_def(dataset_m, dataset_q, transform_m, transform_q)
            [transform_dataset_m, transform_dataset_q, appendix_md, appendix_qd] = rename_columns(transform_dataset_m, transform_dataset_q)
            groups = sort_groups(appendix_md)
            [factors, factor_orders, factor_multiplicities] = load_factors(groups)
            results = run_model(transform_dataset_m, transform_dataset_q, factors, factor_orders, factor_multiplicities)
            
            f = hope(results, transform_dataset_q, date, dataset_q)
            forecast_final.append(f)

            # at end
            count = count + 1
            if count == 3:
                prev_date = date
            

2010-01 2009-12


/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/pandas/core/frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/statsmodels/tsa/statespace/dynamic_factor_mq.py:1947: UserWarning: Non-stationary starting factor autoregressive parameters found for factor block ('Global.1', 'Global.2'). Using zeros as starting parameters.
  warn('Non-stationary starting factor autoregressive'


2010-02 2009-12


/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/pandas/core/frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/statsmodels/tsa/statespace/dynamic_factor_mq.py:1947: UserWarning: Non-stationary starting factor autoregressive parameters found for factor block ('Global.1', 'Global.2'). Using zeros as starting parameters.
  warn('Non-stationary starting factor autoregressive'


2010-03 2009-12


...

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/pandas/core/frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


2022-05 2022-03


/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/pandas/core/frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


SystemExit: 

/Users/tommyflaim/Library/Python/3.9/lib/python/site-packages/IPython/core/interactiveshell.py:3449: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [14]:
df = pd.DataFrame(forecast_final)
df

,0
0,15273.158633
1,15224.168291
2,15259.249593
3,15301.854211
4,15357.736105
...,...
144,19057.215279
145,18962.548450
146,19024.889976
147,18813.453281


In [25]:
df.to_csv('gdp_prediction.csv')